## Run on HARDAC

In [5]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



In [6]:
ls ${FD_RES}

annotation_fragment  count_fragment  coverage  data  log  model_linear  source


In [7]:
ls ${FD_RES}/annotation_fragment

filter_motif_score095  Input4                    TFX2_Dex   TFX4_Dex
Input1                 Input5                    TFX2_DMSO  TFX4_DMSO
Input2                 target_PER1_MA0099.3.bed  TFX3_Dex   TFX5_Dex
Input3                 target_PER1_MA0113.3.bed  TFX3_DMSO  TFX5_DMSO


In [8]:
dir ${FD_RES}/annotation_fragment/Input?/chr21.bed

2.7G Sep  1 17:19 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input1/chr21.bed
2.9G Sep  1 14:54 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input2/chr21.bed
3.4G Sep  1 04:56 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input3/chr21.bed
2.7G Sep  1 14:16 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input4/chr21.bed
2.2G Sep  1 23:24 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/Input5/chr21.bed
13.9


In [9]:
dir ${FD_RES}/annotation_fragment/TFX?_DMSO/chr21.bed

6.3G Sep  1 15:49 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX2_DMSO/chr21.bed
4.1G Sep  1 03:38 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX3_DMSO/chr21.bed
4.8G Sep  1 13:41 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX4_DMSO/chr21.bed
4.5G Sep  1 21:46 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX5_DMSO/chr21.bed
19.7


In [11]:
dir ${FD_RES}/annotation_fragment/TFX?_Dex/chr21.bed

6.7G Sep 10 18:40 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX2_Dex/chr21.bed
4.2G Sep  1 16:42 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX3_Dex/chr21.bed
5.6G Sep  1 22:44 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX4_Dex/chr21.bed
6.8G Sep  1 02:40 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/TFX5_Dex/chr21.bed
23.3


In [12]:
ls ${FD_RES}/annotation_fragment/filter_motif_score095

Input1  Input3  Input5    TFX2_DMSO  TFX3_DMSO  TFX4_DMSO  TFX5_DMSO
Input2  Input4  TFX2_Dex  TFX3_Dex   TFX4_Dex   TFX5_Dex


In [13]:
dir ${FD_RES}/annotation_fragment/filter_motif_score095/Input?/chr21.bed.gz

6.9M Sep 14 14:03 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1/chr21.bed.gz
7.3M Sep 14 14:03 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input2/chr21.bed.gz
8.2M Sep 14 14:04 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input3/chr21.bed.gz
7.0M Sep 14 14:05 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input4/chr21.bed.gz
5.9M Sep 14 14:05 /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input5/chr21.bed.gz
35.3


## Run interaction

In [28]:
cat > motif_interaction.R << 'EOF'
### set env
source("/home/mount/project/config_sing.R")
cat("###################################################\n\n")

### Get argument: Chromomsome
ARGS  = commandArgs(trailingOnly=TRUE)
CHROM = ARGS[1]
print(CHROM)

###################################################
# Import library size
###################################################
cat("+++++ Import library size +++++\n")

### Helper function to get
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    mutate(Sample = tools::file_path_sans_ext(basename(Fpath))) %>%
    mutate(Group = get_sample(Sample))
dat_lib = dat_lib %>% dplyr::select(Size, Sample, Group)

###################################################
# Import annotated fragments
###################################################
cat("+++++ Import annotated fragments +++++\n")

### set column names and types
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

### set samples
SAMPLES = c(
    paste0("Input", 1:5),
    paste0("TFX",   2:5, "_DMSO"),
    paste0("TFX",   2:5, "_Dex"))

### import bed files for each sample 
fdiry = file.path(FD_RES, "annotation_fragment", "filter_motif_score095")
fname = paste0(CHROM, ".bed", ".gz") 

lst_dat = lapply(SAMPLES, function(sam){
    ### set path
    fpath = file.path(fdiry, sam, fname)
    print(fpath)
    
    ### import data
    dat = read_tsv(fpath, col_types=ctypes, col_names=cnames) %>% mutate(Sample = sam)
    return(dat)
})

### arrange data & clean memory
dat_ann_frag = bind_rows(lst_dat)
lst_dat = NULL

### check point: print message
head(dat_ann_frag, 3)
print(colnames(dat_ann_frag))

print("Dimension")
print(dim(dat_ann_frag))

print("Size")
print(object.size(dat_ann_frag), units="Mb")

###################################################
# Preprocess
###################################################

### grouped by motif cluster and 
### split the annotated fragments into list
cat("+++++ Preprocess: get motifs +++++\n")
dat    = dat_ann_frag
lst    = dat %>% group_by(Motif) %>% group_split
motifs = lapply(lst, function(dat){unique(dat$Motif)}) %>% unlist
names(lst) = motifs

### get the list
cat("+++++ Preprocess: get lst_frag +++++\n")
lst_frag = lapply(lst, function(dat){
    tmp = dat %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Motif, Sample) %>%
        distinct()
    return(tmp)
})

### filter out motifs that have almost no fragment in total
cat("+++++ Preprocess: filter motifs +++++\n")
cat("Before filteration: #Motifs =", length(lst_frag), "\n")

THRESHOLD = 10
lst = lst_frag
cnt = lapply(lst, function(dat){sum(dat$Count_Frag)})
lst = lst[cnt > 10]
lst_frag = lst

cat("Threshold =", THRESHOLD, "\n")
cat("After filteration: #Motifs =", length(lst_frag), "\n")

###################################################
# Set all pairs of motifs
###################################################
cat("+++++ Set all pairs of motifs +++++\n")

### combination of motifs
dat_comb = t(combn(names(lst_frag), 2))

### convert motif pairs into a list
lst_motif_pair = split(dat_comb, seq(nrow(dat_comb)))
names(lst_motif_pair) = lapply(
    lst_motif_pair, function(x){
        mtf1 = x[1]
        mtf2 = x[2]
        return(paste(mtf1, mtf2, sep="|"))
    } # end fun
) # end lapply

###################################################
# Get interactive effects
###################################################
cat("+++++ Get interactive effects +++++\n")

### Helper function
### https://win-vector.com/2014/05/30/trimming-the-fat-from-glm-models-in-r/
stripGlmLR = function(cm) {
  cm$y = c()
  cm$model = c()
  
  cm$residuals = c()
  cm$fitted.values = c()
  cm$effects = c()
  cm$qr$qr = c()  
  cm$linear.predictors = c()
  cm$weights = c()
  cm$prior.weights = c()
  cm$data = c()

  cm$family$variance = c()
  cm$family$dev.resids = c()
  cm$family$aic = c()
  cm$family$validmu = c()
  cm$family$simulate = c()
  attr(cm$terms,".Environment") = c()
  attr(cm$formula,".Environment") = c()
  
  return(cm)
}

### loop through each pair of motifs
### estimate interaction effect of each motif pair
lst_res = lapply(lst_motif_pair, function(x){

    ### extract fragments for each motif
    mtf1 = x[1]
    mtf2 = x[2]
    df1 = lst_frag[[mtf1]]
    df2 = lst_frag[[mtf2]]
    
    ### extract fragments
    dat1 = bind_rows(df1, df2) %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample) %>%
        distinct
    dat2 = df1 %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample, Motif)
    dat3 = df2 %>% 
        dplyr::select(Chrom_Frag, Start_Frag, End_Frag, Count_Frag, Sample, Motif)
    
    ### match fragments for the motif pair
    dat = dat1 %>%
        full_join(dat2, by = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Sample")) %>%
        full_join(dat3, by = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag", "Sample")) %>%
        mutate(Motif = paste(Motif.x, Motif.y, sep = "_")) %>%
        mutate(Motif = str_remove(string=Motif, pattern="_NA|NA_")) %>% 
        mutate(Group = str_remove(string = Sample, pattern = "[0-9]")) %>%
        mutate(X     = paste(Group, Motif, sep="_")) %>%
        mutate(X     = ifelse(str_detect(X, "Input"), "Input", X))
    
    ### annotate fragments based on motif annotation
    idx11 = paste("TFX_DMSO", mtf1,       sep="_")
    idx12 = paste("TFX_DMSO", mtf2,       sep="_")
    idx13 = paste("TFX_DMSO", mtf1, mtf2, sep="_")
    idx21 = paste("TFX_Dex",  mtf1,       sep="_")
    idx22 = paste("TFX_Dex",  mtf2,       sep="_")
    idx23 = paste("TFX_Dex",  mtf1, mtf2, sep="_")
    idxs  = c("Input", idx11, idx12, idx21, idx22, idx13, idx23)
    tmp = dat %>% 
        group_by(Sample, X) %>% 
        summarise(Value = sum(Count_Frag), .groups = 'drop')
    
    ### normalize counts by library size
    tmp = tmp %>% left_join(dat_lib, by="Sample")
    tmp = tmp %>%
        mutate(Norm_Value    = Value / Size) %>% 
        mutate(Lognorm_Value = log10(Value) - log10(Size))
    tmp$X = factor(tmp$X, levels=idxs)
    X = model.matrix(~X, tmp)
    y = tmp$Norm_Value
    
    ### create design matrix
    idx11 = paste("XTFX_DMSO", mtf1,       sep="_")
    idx12 = paste("XTFX_DMSO", mtf2,       sep="_")
    idx13 = paste("XTFX_DMSO", mtf1, mtf2, sep="_")
    idx21 = paste("XTFX_Dex",  mtf1,       sep="_")
    idx22 = paste("XTFX_Dex",  mtf2,       sep="_")
    idx23 = paste("XTFX_Dex",  mtf1, mtf2, sep="_")
    X[,idx11] = X[,idx11] + X[,idx13] + X[,idx21] + X[,idx23]
    X[,idx12] = X[,idx12] + X[,idx13] + X[,idx22] + X[,idx23]
    X[,idx21] = X[,idx21] + X[,idx23]
    X[,idx22] = X[,idx22] + X[,idx23]
    X[,idx13] = X[,idx13] + X[,idx23]
    
    ### fit model and get the summary
    fit = lm(y ~ X + 0)
    res = summary(fit)
    
    ### reduce the memory size
    res = stripGlmLR(res)
    return(res)
})

###################################################
# Save the results
###################################################

fdiry = file.path(FD_RES, "model_linear")
fname = paste0("res_interactive_", CHROM, ".rds") # fname="res_interactive_chr21.rds"
fpath = file.path(fdiry, fname)
saveRDS(lst_res, fpath)
print("Done")

EOF

In [15]:
cat motif_interaction.R

### set env
source("/home/mount/project/config_sing.R")
cat("###################################################\n\n")

### Get argument: Chromomsome
ARGS  = commandArgs(trailingOnly=TRUE)
CHROM = ARGS[1]
print(CHROM)

###################################################
# Import library size
###################################################
cat("+++++ Import library size +++++\n")

### Helper function to get
get_sample = function(idn_sample){
    idn = idn_sample
    idn = str_replace(
        string = idn, 
        pattern = "Input[0-9]", 
        replacement = "Input")
    idn = str_replace(
        string = idn, 
        pattern = "TFX[0-9]_", 
        replacement="TFX_")
    return(idn)
}

### set path
fdiry = file.path(FD_RES, "source")
fname = "library_size.txt"
fpath = file.path(fdiry, fname)

### import library size
ctypes = c(col_integer(), col_character())
cnames = c("Size", "Fpath")
dat_lib = read_tsv(fpath, col_types=ctypes, col_names = cnames)
dat_lib = dat_lib %>% 
    

In [23]:
echo $NODE
echo $FD_LOG

all
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log


## Run on single chromosome

In [24]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_interaction_095_chrY.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chrY

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R ${CHROM}

EOF

Submitted batch job 26086261


In [27]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_interaction_095_chr20.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr20

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R ${CHROM}

EOF

Submitted batch job 26086264


In [26]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_interaction_095_chr21.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr21

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R ${CHROM}

EOF

Submitted batch job 26086263


In [29]:
### set log file directory
sbatch -p ${NODE} \
    --mem=30G \
    -o ${FD_LOG}/motif_interaction_095_chr22.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr22

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R ${CHROM}

EOF

Submitted batch job 26086265


In [39]:
echo $NODE

all


In [40]:
### set log file directory
sbatch -p ${NODE} \
    --mem=50G \
    -o ${FD_LOG}/motif_interaction_095_chr17.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
CHROM=chr17

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R ${CHROM}

EOF

Submitted batch job 26098575


-----

## Run all at the same time

In [93]:
### set log file directory
sbatch -p ${NODE} \
    --array=0-23 \
    --mem=20G \
    -o ${FD_LOG}/motif_interaction_095.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}

###
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R ${CHROM}

EOF

Submitted batch job 26083009


scancel 26083009

#./watching.sh /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/motif_interaction_095.0.txt
#./watching.sh /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/motif_interaction_095.23.txt

In [76]:
cat ${FD_LOG}/motif_interaction_095.0.txt

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine

###################################################

[1] "chr1"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1/chr1.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input2/chr1.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input3/chr1.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input4/chr1.bed.gz"
[1] "/home/mount/work/out/proj_combef

In [18]:
cat ${FD_LOG}/motif_interaction_095.13.txt

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine

###################################################

[1] "chr14"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1/chr14.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input2/chr14.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input3/chr14.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input4/chr14.bed.gz"
[1] "/home/mount/work/out/proj_c

In [38]:
cat ${FD_LOG}/motif_interaction_095.22.txt

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine

###################################################

[1] "chrX"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input1/chrX.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input2/chrX.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input3/chrX.bed.gz"
[1] "/home/mount/work/out/proj_combeffect/annotation_fragment/filter_motif_score095/Input4/chrX.bed.gz"
[1] "/home/mount/work/out/proj_combef

In [30]:
ls ${FD_RES}/model_linear

fit_interactive_target_PER1.rds  res_interactive_chrX.rds
fit_marginal_target_PER1.rds     res_interactive_chrY.rds
res_interactive_chr20.rds        res_interactive_target_PER1.rds
res_interactive_chr21.rds


## Run on DCC

In [45]:
readlink -f ../config_duke.sh

/hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/config_duke.sh


In [3]:
ls ${FD_RES}

annotation_fragment  bedgraph   count_fragment  coverage  kmer  model_linear
annotation_segment   bootstrap  count_segment   data      log   source


In [46]:
ls ${FD_RES}/annotation_fragment

Input   Input3  target_PER1_MA0099.3.bed  TFX2_DMSO  TFX4_Dex   TFX5_DMSO
Input1  Input4  target_PER1_MA0113.3.bed  TFX3_Dex   TFX4_DMSO  TFX_Dex
Input2  Input5  TFX2_Dex                  TFX3_DMSO  TFX5_Dex   TFX_DMSO


In [43]:
dir ${FD_RES}/annotation_fragment/Input?/chr21.bed

2.7G Jul 19 00:13 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr21.bed
2.9G Jul 19 00:15 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/chr21.bed
3.4G Jul 19 00:18 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/chr21.bed
2.7G Jul 19 00:19 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/chr21.bed
2.2G Jul 19 00:21 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/chr21.bed
13.9


In [44]:
dir ${FD_RES}/annotation_fragment/TFX?_DMSO/chr21.bed

6.3G Jul 19 00:26 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/chr21.bed
4.1G Jul 19 00:33 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_DMSO/chr21.bed
4.8G Jul 19 00:39 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_DMSO/chr21.bed
4.5G Jul 19 00:46 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_DMSO/chr21.bed
19.7


In [37]:
dir ${FD_RES}/annotation_fragment/TFX?_Dex/chr21.bed

6.7G Jul 19 00:31 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/chr21.bed
4.2G Jul 19 00:36 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_Dex/chr21.bed
5.6G Jul 19 00:43 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_Dex/chr21.bed
6.8G Jul 19 00:50 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_Dex/chr21.bed
23.3


In [38]:
python -c "print(2.7 + 2.9 + 3.4 + 2.7 + 2.2)"
python -c "print(13.9 + 19.7 + 23.3)"

13.899999999999999
56.900000000000006


In [4]:
dir ${FD_RES}/annotation_fragment/Input?/chr17.bed

7.1G Jul 19 00:30 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr17.bed
7.7G Jul 19 00:35 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input2/chr17.bed
9.0G Jul 19 00:42 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input3/chr17.bed
7.2G Jul 19 00:46 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input4/chr17.bed
5.8G Jul 19 00:50 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input5/chr17.bed
36.8


In [5]:
dir ${FD_RES}/annotation_fragment/TFX?_DMSO/chr17.bed

19G Jul 19 01:01 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_DMSO/chr17.bed
12G Jul 19 01:18 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_DMSO/chr17.bed
14G Jul 19 01:32 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_DMSO/chr17.bed
13G Jul 19 01:48 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_DMSO/chr17.bed
58


In [6]:
dir ${FD_RES}/annotation_fragment/TFX?_Dex/chr17.bed

19G Jul 19 01:12 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX2_Dex/chr17.bed
12G Jul 19 01:25 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX3_Dex/chr17.bed
16G Jul 19 01:41 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX4_Dex/chr17.bed
19G Jul 19 01:58 /work/kk319/out/CombEffect_STARR/annotation_fragment/TFX5_Dex/chr17.bed
66


In [7]:
python -c "print(7.1 + 7.7 + 9.0 + 7.2 + 5.8)"

36.8


In [8]:
python -c "print(36.8 + 58 + 66)"

160.8


In [9]:
${FD_PRJ}/sing_proj_combeffect.sh ls

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation
PATH OF PROJECT:    /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /work/kk319/out/CombEffect_STARR

01_marginal_per1_v1.ipynb	  12_interactive_per1_multiple.ipynb  config.R
01_marginal_per1_v2.ipynb	  13_interactive_per1_plot.ipynb      mount
11_interactive_per1_single.ipynb  14_interactive_chr17.ipynb


In [10]:
${FD_PRJ}/sing_proj_combeffect.sh ls ./mount

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation
PATH OF PROJECT:    /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /work/kk319/out/CombEffect_STARR

project  reddylab  work


In [12]:
${FD_PRJ}/sing_proj_combeffect.sh bash << 'EOF'
#!/bin/bash
source ./mount/project/config_sing.sh
ls ${FD_WORK}
echo ++++++++++++++++++++++++++++
ls ${FD_RLAB}
EOF

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation
PATH OF PROJECT:    /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /work/kk319/out/CombEffect_STARR

You are within a singularity image
BASE DIRECTORY:     /home/mount/work
PATH OF SOURCE:     /home/mount/work/source
PATH OF EXECUTABLE: /home/mount/work/exe
PATH OF ANNOTATION: /home/mount/work/annotation
PATH OF PROJECT:    /home/mount/project
PATH OF RESULTS:    /home/mount/work/out/CombEffect_STARR

annotation  exe  out  source
++++++++++++++++++++++++++++
CRADLE	Kuei  scarlett


In [13]:
ls ${FD_RES}

annotation_fragment  bedgraph   count_fragment  coverage  kmer  model_linear
annotation_segment   bootstrap  count_segment   data      log   source


In [16]:
ls ${FD_LOG} | head

annotate_fragment_chrom.0.txt
annotate_fragment_chrom.10.txt
annotate_fragment_chrom.11.txt
annotate_fragment_chrom.12.txt
annotate_fragment_chrom.13.txt
annotate_fragment_chrom.14.txt
annotate_fragment_chrom.15.txt
annotate_fragment_chrom.16.txt
annotate_fragment_chrom.17.txt
annotate_fragment_chrom.18.txt
ls: write error: Broken pipe


In [19]:
#NODE_NAME=scavenger
echo ${NODE_NAME}

scavenger


In [32]:
head -3 /work/kk319/out/CombEffect_STARR/annotation_fragment/Input1/chr17.bed

chr17	201123	201925	1	chr17	201116	201126	NFKB/1	9.0328	3
chr17	201123	201925	1	chr17	201136	201152	MZF1	9.0967	16
chr17	201123	201925	1	chr17	201138	201148	NFKB/1	8.1413	10


In [33]:
cat > motif_interaction.R << 'EOF'
### set env
source("/home/mount/project/config_sing.R")
#print(dir(FD_WORK))
#print(dir(FD_RES))

### import
ctypes = c(col_character(), col_integer(), col_integer(), col_integer(),
           col_character(), col_integer(), col_integer(),
           col_character(), col_double(),  col_integer())
cnames = c("Chrom_Frag", "Start_Frag", "End_Frag", "Count_Frag",
           "Chrom_MTF",  "Start_MTF",  "End_MTF",
           "Motif", "Score", "Overlap")

fdiry = file.path(FD_RES, "annotation_fragment", "Input1")
fname = "chr17.bed"
fpath = file.path(fdiry, fname)

dat = read_tsv(fpath, col_types=ctypes, col_names=cnames)
print(head(dat, 3))

EOF

In [34]:
### set log file directory
sbatch -p ${NODE_NAME} \
    --mem=10G \
    -o ${FD_LOG}/motif_interaction_chr17.txt \
    <<'EOF'
#!/bin/bash
source ../config_duke.sh
${FD_PRJ}/sing_proj_combeffect.sh Rscript motif_interaction.R

EOF

Submitted batch job 17810368


In [23]:
echo ${FD_LOG}/motif_interaction_chr17.txt

/work/kk319/out/CombEffect_STARR/log/motif_interaction_chr17.txt


In [93]:
cat ${FD_LOG}/test.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation
PATH OF PROJ:       /hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF OUTPUT:     /work/kk319/out/CombEffect_STARR

/hpc/home/kk319/GitRepo/Proj_CombEffect_STARRseq/notebooks/sing_proj_combeffect.sh: line 3: config_duke.sh: No such file or directory
Error for command "exec": requires at least 2 arg(s), only received 1

Usage:
  singularity [global options...] exec [exec options...] <container> <command>

Run 'singularity --help' for more detailed usage information.


In [51]:
ls -lh ${FD_RES}/annotation_fragment/motif_annotation_chr21_090.bed

-rw-r--r--. 1 kk319 root 4.8G Sep  6 12:43 /work/kk319/out/CombEffect_STARR/annotation_fragment/motif_annotation_chr21_090.bed


In [50]:
ls -lh ${FD_RES}/annotation_fragment/motif_annotation_chr21_095.bed

-rw-r--r--. 1 kk319 root 1.9G Sep  6 11:29 /work/kk319/out/CombEffect_STARR/annotation_fragment/motif_annotation_chr21_095.bed
